# Convolutional Neural Network

### Part 1 - Building the CNN

#### Importing the Tensorflow libraries and packages

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tqdm import tqdm
import os

In [2]:
TRAIN_DIR = 'dataset/train'
TEST_DIR = 'dataset/test'
VALID_DIR = 'dataset/valid'
IMG_SIZE = 255 
LR = 1e-3 #learn rate, quanto menor mais aprende

In [3]:
MODEL_NAME = 'CNN-{}-{}.model'.format(LR, 'Xconv-basic')

In [4]:
classifier = Sequential()

#classes = 250

#Kernels usually 1x1 | 3x3 | 5x5

#Convolution + MaxPooling layer
# Arguments: 32 filters, 3x3 kernel, relu activation
classifier.add(Conv2D(32, (3, 3), input_shape = (IMG_SIZE, IMG_SIZE, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

#Adding a second convolutional layer
classifier.add(Conv2D(64, (3, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

#Adding a third convolutional layer
classifier.add(Conv2D(128, (3, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

#Adding a fourth convolutional layer
classifier.add(Conv2D(256, (3, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Flatten())

classifier.add(Dense(units = 1024, activation = 'relu'))
#classifier.add(Dropout(0.5))
classifier.add(Dense(units = 250, activation = 'softmax'))

Let's double check the model description:

In [5]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 255, 255, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 127, 127, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 63, 63, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 31, 31, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 31, 31, 256)       2

In [6]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Part 2 - Fitting the CNN to the images


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./IMG_SIZE,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./IMG_SIZE)

validation_datagen = ImageDataGenerator(rescale = 1./IMG_SIZE)

In [8]:
training_set = train_datagen.flow_from_directory(TRAIN_DIR,
                                                 target_size = (IMG_SIZE, IMG_SIZE),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(TEST_DIR,
                                            target_size = (IMG_SIZE, IMG_SIZE),
                                            batch_size = 32,
                                            class_mode = 'categorical')

validation_set = validation_datagen.flow_from_directory(VALID_DIR,
                                            target_size = (IMG_SIZE, IMG_SIZE),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 35215 images belonging to 250 classes.
Found 1250 images belonging to 250 classes.
Found 1250 images belonging to 250 classes.


In [9]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from datetime import datetime

es = EarlyStopping(monitor='val_loss', patience=5)
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)

# Create a TensorBoard callback
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard = TensorBoard(log_dir = logs,
                          histogram_freq = 1,
                          profile_batch = '500,520')

In [10]:
classifier.fit( training_set,
                         epochs = 25,
                         callbacks = [es, mc, tensorboard],
                         validation_data = validation_set,
                         workers = 4)

Epoch 1/25
1101/1101 [==============================] - 1672s 2s/step - loss: 0.0360 - accuracy: 0.0203 - val_loss: 0.0175 - val_accuracy: 0.2528
Epoch 2/25
1101/1101 [==============================] - 1703s 2s/step - loss: 0.0168 - accuracy: 0.2965 - val_loss: 0.0108 - val_accuracy: 0.5520
Epoch 3/25
1101/1101 [==============================] - 1703s 2s/step - loss: 0.0117 - accuracy: 0.5233 - val_loss: 0.0084 - val_accuracy: 0.6768
Epoch 4/25
1101/1101 [==============================] - 1656s 2s/step - loss: 0.0091 - accuracy: 0.6399 - val_loss: 0.0077 - val_accuracy: 0.7048
Epoch 5/25
1101/1101 [==============================] - 1665s 2s/step - loss: 0.0072 - accuracy: 0.7342 - val_loss: 0.0066 - val_accuracy: 0.7624
Epoch 6/25
1101/1101 [==============================] - 1674s 2s/step - loss: 0.0058 - accuracy: 0.7920 - val_loss: 0.0064 - val_accuracy: 0.7688
Epoch 7/25
1101/1101 [==============================] - 1645s 1s/step - loss: 0.0048 - accuracy: 0.8382 - val_loss: 0.0064 -